# Predicting Car Make with CNNs

## Overview

The purpose of this notebook is self-learning. It's an attempt to predict car manufacturer using a convolutional neural network.

## Setup

TensorFlow 2 will be used to build the neural network, and it will be accessed via the Keras front-end.

In [1]:
from glob import glob
from PIL import Image
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras import layers
from keras import models
from keras import backend
from keras import optimizers
import pandas as pd
import numpy as np
import os
os.chdir('/Users/markroepke/Documents/fun/projects/car-images/')

Using TensorFlow backend.


## Data Prep

The entire data set is made up of 64,000 images of cars from [thecarconnection.com](). It was originally sourced by Nicolas Gervais, and details can be found on his Reddit post: https://www.reddit.com/r/MachineLearning/comments/ek5zwv/p_64000_pictures_of_cars_labeled_by_make_model/.

For now, the data is not cross-validated so it's simply split into training, validating, and testing data sets.

In [3]:
# Load and randomize all image file names
full_image_paths = glob("data/full-images/*.jpg")
full_image_paths = np.random.permutation(full_image_paths)

# Process each image and its label
scaled_images = []
makes = []
for idx, image_path in enumerate(full_image_paths):
    if idx > 10000:
        break
        
    # Resize each image to 256x256 and convert it to grayscale
    processed_image = Image.open(image_path).resize((256, 256)).convert('L')
    scaled_images.append(np.array(processed_image))
    
    # Get the make of the car from the file name
    makes.append(image_path.split("/")[2].split("_")[0])
    
    # Print update as this can take a little while
    if idx % round(len(full_image_paths) / 10) == 0:
        print(f"Finished {idx} photos, {round((idx / len(full_image_paths)) * 100)}%")
        
# Rescale images for keras/tensorflow
scaled_images_array = np.array(scaled_images)
scaled_images_array = scaled_images_array.reshape(
    scaled_images_array.shape[0], 
    scaled_images_array.shape[1], 
    scaled_images_array.shape[2],
    1
)

# Get number of distant labels present in data set
distinct_y = len(set(list(makes)))

# Transform labels for keras/tensorflow
encoder = LabelBinarizer()
makes_array = encoder.fit_transform(np.array(makes))

# Create x datasets
train_x = scaled_images_array[:round(0.7 * len(scaled_images_array))]
val_x = scaled_images_array[round(0.7 * len(scaled_images_array)):round(0.9 * len(scaled_images_array))]
holdout_x = scaled_images_array[round(0.9 * len(scaled_images_array)):]

# Create y datasets
train_y = makes_array[:round(0.7 * len(makes_array))]
val_y = makes_array[round(0.7 * len(makes_array)):round(0.9 * len(makes_array))]
holdout_y = makes_array[round(0.9 * len(makes_array)):]

Finished 0 photos, 0%
Finished 6447 photos, 10%


# Model

In [4]:
# Clear TensorFlow session for iterative development
backend.clear_session()

# Build convolutional layers for vision
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape = (256, 256, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten convolutional output and condense into predictions
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(distinct_y, activation = 'softmax'))

# Compile model
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics = ['acc'])

# Train model
model.fit(x = train_x, y = train_y, epochs = 10, batch_size = 50, validation_data = (val_x, val_y))

Train on 7001 samples, validate on 2000 samples
Epoch 1/10
7001/7001 [==============================] - 291s 42ms/step - loss: 0.2709 - acc: 0.9697 - val_loss: 0.2309 - val_acc: 0.9533
Epoch 2/10
7001/7001 [==============================] - 288s 41ms/step - loss: 0.1047 - acc: 0.9760 - val_loss: 0.1235 - val_acc: 0.9684
Epoch 3/10
7001/7001 [==============================] - 289s 41ms/step - loss: 0.0786 - acc: 0.9786 - val_loss: 0.1880 - val_acc: 0.9564
Epoch 4/10
7001/7001 [==============================] - 289s 41ms/step - loss: 0.0497 - acc: 0.9848 - val_loss: 0.2366 - val_acc: 0.9586
Epoch 5/10
7001/7001 [==============================] - 311s 44ms/step - loss: 0.0279 - acc: 0.9912 - val_loss: 0.1873 - val_acc: 0.9655
Epoch 6/10
7001/7001 [==============================] - 288s 41ms/step - loss: 0.0152 - acc: 0.9954 - val_loss: 0.1566 - val_acc: 0.9756
Epoch 7/10
7001/7001 [==============================] - 299s 43ms/step - loss: 0.0091 - acc: 0.9975 - val_loss: 0.1948 - val_acc: 

# Evaluate on Holdout Set

Evaluate on a true holdout set for final performance metric --> about 98% accuracy.

In [8]:
holdout_loss, holdout_acc = model.evaluate(holdout_x, holdout_y)

1000/1000 [==============================] - 12s 12ms/step


In [9]:
holdout_acc

0.9771905541419983